In [1]:
import os
import cv2
import numpy as np

import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor
from scipy.optimize import minimize

from keras.applications.mobilenet_v2 import MobileNetV2

In [2]:
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth")
sam.to(device="cuda")

predictor = SamPredictor(sam)

In [4]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

# Creación de los generadores de imágenes, haz data augmentation en el conjunto de entrenamiento un 20% de las imágenes (albumentations)

MAIN_PATH = 'Final_dataset_small'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    #preprocessing_function=transform
)

train_generator = train_datagen.flow_from_directory(
    os.path.join(MAIN_PATH, 'Train'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    os.path.join(MAIN_PATH, 'Validation'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 48636 images belonging to 10 classes.
Found 4700 images belonging to 10 classes.


In [3]:
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Activation, Input, Layer

class MyCustomLayer(Layer):
    def __init__(self, points):
        super(MyCustomLayer, self).__init__()
        self.points = points

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)


# Create movile net model
cnn_sam = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
cnn_prd = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a model with two inputs (Funcional API)
input_img1 = Input(shape=(224, 224, 3))

cnn_sam = cnn_sam(input_img1)
mlp_sam = Flatten()(cnn_sam)
mlp_sam = Dense(512, activation='relu')(mlp_sam)
mlp_sam = Dropout(0.5)(mlp_sam)
mlp_sam = Dense(256, activation='relu')(mlp_sam)
points = Dense(4, activation='relu')(mlp_sam)

input_img2 = Input(shape=(224, 224, 3))

cnn_prd = cnn_prd(input_img2)
mlp_prd = Flatten()(cnn_prd)
mlp_prd = Dense(1024, activation='relu')(mlp_prd)
mlp_prd = Dropout(0.5)(mlp_prd)
mlp_prd = Dense(10, activation='softmax')(mlp_prd)






